In [1]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 6.3 MB/s eta 0:00:0000:0100:01


In [2]:
import duckdb

In [16]:
import httpx
from typing import Optional

In [17]:
async def fetch(url: str, client: Optional[httpx.AsyncClient] = None, **kwargs) -> dict:
    """From here: https://stackoverflow.com/questions/22190403/how-could-i-use-requests-in-asyncio/50312981#50312981"""
    if client:
        response = await client.get(url, **kwargs)
    else:
        async with httpx.AsyncClient() as client:
            response = await client.get(url, **kwargs)
    return response.json()

In [18]:
await fetch(
    f"https://collectionapi.metmuseum.org/public/collection/v1/search?hasImages=true&q=landscape"

)

{'total': 11393,
 'objectIDs': [359362,
  10181,
  11978,
  48968,
  11231,
  11181,
  19334,
  10560,
  11630,
  437323,
  11774,
  436455,
  250945,
  250939,
  250946,
  282,
  250930,
  9620,
  334002,
  36131,
  342003,
  342191,
  342046,
  347917,
  72416,
  45650,
  49614,
  438638,
  75680,
  36438,
  337306,
  76393,
  49055,
  459968,
  75634,
  44895,
  339732,
  49451,
  335112,
  690986,
  49455,
  54710,
  363370,
  339645,
  440329,
  333776,
  339790,
  75612,
  438622,
  51610,
  51613,
  751593,
  386043,
  11532,
  356890,
  51502,
  337807,
  357070,
  19252,
  72603,
  267594,
  334798,
  489337,
  829383,
  49013,
  364055,
  36293,
  720026,
  719895,
  719965,
  719966,
  719939,
  719961,
  719950,
  767759,
  36364,
  36365,
  36312,
  385236,
  54437,
  44581,
  392991,
  36306,
  36350,
  36308,
  374518,
  671033,
  765254,
  765250,
  10660,
  10659,
  12844,
  12838,
  394015,
  372742,
  339872,
  359262,
  335325,
  15973,
  40426,
  51481,
  51503,
  

In [19]:
test = await fetch(
                f"https://collectionapi.metmuseum.org/public/collection/v1/objects/480998"
            )

In [22]:
import fs

ModuleNotFoundError: No module named 'fs'

In [21]:
duckdb.sql(
    """
select 
* 
from read_json(test)
    """).show()

IOException: IO Error: No files found that match the pattern "test"